# Location

The `location` directive, used within the `server` context, specifies how NGINX should process requests based on the URI.

More in [correspoding section](http://nginx.org/en/docs/http/ngx_http_core_module.html#location) of the official documentation.

In [1]:
import os
import docker
import requests

docker_client = docker.DockerClient()

## Setup

It turns out that it's quite a complex task to build examples that show how everything works, so this section describes what we need to show everything. In summary, we need

- Proxied server - server where we'll redirect requests to nginx.
- And nginx, which can be configured differently for different examples.

### Network

We need a way to connect containers between each other - so we're going to create a network that will be used for container communication.

In [2]:
network = docker_client.networks.create(name = "test_network")

### Proxied server

Here is the simplest possible http server in Python. It's purpose is just to return it's input as raw text - so we can check what was sent to the server.

In [3]:
%%writefile location_files/proxy_set_header.py
import http.server
import socketserver

class RequestHandler(http.server.BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.send_header("Content-type", "text/plain")
        self.end_headers()
        self.wfile.write(b"Server received your GET request\n")
        self.wfile.write(b"Raw request data:\n\n")
        self.wfile.write(self.raw_requestline)
        self.wfile.write(b"\nHeaders:\n")
        self.wfile.write(bytes(str(self.headers), "utf-8"))

PORT = 7890
with socketserver.TCPServer(("", PORT), RequestHandler) as httpd:
    print("Serving at port", PORT)
    httpd.serve_forever()

Overwriting location_files/proxy_set_header.py


For standardisation purposes, we'll run this server in the docker container.

In [4]:
client_container = docker_client.containers.run(
    image = "python:3.10-alpine",
    volumes = {
        f"{os.getcwd()}/location_files/proxy_set_header.py": 
        {'bind': '/proxy_set_header.py', 'mode': 'rw'}
    },
    command = "python proxy_set_header.py",
    ports = {7890: 7890},
    detach = True,
    remove = True,
    network = network.name,
    name = "client_container"
)

The next cell shows how it works - it sends a request to the server we created. And for example, we'll add some headers to the request.

In [5]:
print(
    requests.get(
        "http://localhost:7890", 
        headers={"my_field": "my_value"}
    ).content.decode("utf-8")
)

Server received your GET request
Raw request data:

GET / HTTP/1.1

Headers:
Host: localhost:7890
User-Agent: python-requests/2.31.0
Accept-Encoding: gzip, deflate
Accept: */*
Connection: keep-alive
my_field: my_value




As a result we got raw GET HTTP request and there is defined by us header field in the output.

### Nginx

Now we configure nginx. It's config can be changed, but by default it will forward requests to it root to the http server we created earlier.

In [6]:
%%writefile location_files/nginx.conf
events {}
http {
    server {
        listen 80;
        location / {proxy_pass "http://client_container:7890";}
    }
}

Overwriting location_files/nginx.conf


And we run docker container with nginx on the same network as our client http server.

In [7]:
nginx_container = docker_client.containers.run(
    image = "nginx",
    volumes = {
        f"{os.getcwd()}/location_files/nginx.conf": 
        {'bind': '/etc/nginx/nginx.conf', 'mode': 'rw'}
    },
    ports = {80: 80},
    detach = True,
    remove = True,
    network = network.name
)

Now let's see how it works by sending a request to the `localhost:80` that nginx will proxy to the http server.

In [8]:
print(
    requests.get(
        "http://localhost:80", 
        headers={"my_field": "my_value"}
    ).content.decode("utf-8")
)

Server received your GET request
Raw request data:

GET / HTTP/1.0

Headers:
Host: client_container:7890
Connection: close
User-Agent: python-requests/2.31.0
Accept-Encoding: gzip, deflate
Accept: */*




We got the same output as if we had just requested the http server.

### Clear environment

To avoid a lot of rubbish in your environment, you need to delete the containers you have created.

In [9]:
client_container.stop()
nginx_container.stop()
network.remove()

## Headers to proxied server (`proxy_set_header`)

Allows redefining or appending fields to the request header passed to the proxied server. You can add aditional headers to the http request that will be sent to the destination server. So by using syntax `proxy_set_header <header field> <value>;`.

The following example changes the nginx config to add two new fields to the http header `Name` and `SecondName`.

In [13]:
%%writefile location_files/nginx.conf
events {}
http {
    server {
        listen 80;
        location / {
            proxy_pass "http://client_container:7890";
            proxy_set_header Name Fedor;
            proxy_set_header SecondName Kobak;
        }
    }
}

Overwriting location_files/nginx.conf


After reloading nginx, we need to try sending the request to the original server and then to nginx.

In [18]:
ans = nginx_container.exec_run("nginx -s reload")

print("=====Original request=====")
print(
    requests.get("http://localhost:7890").content.decode("utf-8")
)
print("=====Proxied request=====")
print(
    requests.get("http://localhost:80").content.decode("utf-8")
)

=====Original request=====
Server received your GET request
Raw request data:

GET / HTTP/1.1

Headers:
Host: localhost:7890
User-Agent: python-requests/2.31.0
Accept-Encoding: gzip, deflate
Accept: */*
Connection: keep-alive


=====Proxied request=====
Server received your GET request
Raw request data:

GET / HTTP/1.0

Headers:
Name: Fedor
SecondName: Kobak
Host: client_container:7890
Connection: close
User-Agent: python-requests/2.31.0
Accept-Encoding: gzip, deflate
Accept: */*




If you compare the output of the original and proxied requests, you can see that the proxied requests have additional headers - just as we specified in the nginx configuration.